In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from tueplots import bundles
import matplotlib.pyplot as plt
import re
import os

In [ ]:
plt.rcParams.update(bundles.neurips2021())

In [ ]:
dat_src = r'../dat/bundesnetzagentur/pre_processed_data/prod_price_de_lu_2019_2021.csv'
fig_src = r'../doc/fig/'

In [ ]:
combined = pd.read_csv(dat_src,
                       parse_dates=['time'])
combined = combined.set_index('time')
combined = combined.dropna()

In [ ]:
def to_tex(text):
    """
    Helper function to prepare string for latex.
    """
    conv = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde{}',
        '^': r'\^{}',
        '\\': r'\textbackslash{}',
        '<': r'\textless{}',
        '>': r'\textgreater{}',
    }
    regex = re.compile('|'.join(re.escape(str(key)) for key in sorted(conv.keys(), key = lambda item: - len(item))))
    return regex.sub(lambda match: conv[match.group()], text)

# Regression

In [ ]:
cols = [('Solar', 'tab:orange'), ('Wind', 'tab:blue'), ('Solar & Wind', 'tab:green')]

fig, axs = plt.subplots(1, 3, figsize=(12,3), sharey=True) # figsize=(16,4), 

axs[0].set_ylabel('Price [€/MWh]')

# tmp = combined.sample(15000)

for i in range(0, len(axs)):
    col, color = cols[i]
    ax  = axs[i]
    
    X = combined[col].to_numpy().reshape(-1,1)
    y = combined['Price'].to_numpy()

    reg = LinearRegression()
    reg.fit(X, y)

    m = reg.coef_
    b = reg.intercept_
    
    ax.plot(X, y, 'o', color=color, ms=1, alpha=0.5)
    ax.plot(X, m*X + b, 'black')

    ax.set_xlabel(f'{to_tex(col)} Generation [MWh]')

plt.savefig(os.path.join(fig_src, 'solar_wind_regression.pdf'))

## Comparison of Renewable and Conventional

In [ ]:
renewable = ['Biomass', 
             'Hydropower', 
             'Offshore Wind', 
             'Onshore Wind', 
             'Solar', 
             'Other Renewables']
conventional = ['Nuclear', 
                'Brown Coal', 
                'Black Coal',
                'Natural Gas',
                'Other Conventional Sources']

con_ren                 = combined # .resample('D').mean()
con_ren['Conventional'] = con_ren.loc[:, conventional].sum(axis=1)
con_ren['Renewable']    = con_ren.loc[:, renewable].sum(axis=1)
con_ren                 = con_ren.filter(['Conventional', 'Renewable', 'Price'])
con_ren['Total']        = con_ren.loc[:, ('Conventional', 'Renewable')].sum(axis=1)

In [ ]:
X_con = con_ren['Conventional'].to_numpy().reshape(-1,1)
X_ren = con_ren['Renewable'].to_numpy().reshape(-1,1)
y     = con_ren['Price']

reg_con = LinearRegression()
reg_ren = LinearRegression()

reg_con.fit(X_con, y)
reg_ren.fit(X_ren, y)

m_con, b_con = reg_con.coef_, reg_con.intercept_
m_ren, b_ren = reg_ren.coef_, reg_ren.intercept_

plt.figure(figsize=(8,4))

ren_color = 'tab:green'
con_color = 'tab:purple'

plt.plot(X_ren, y, 'o', label='Renewable', color=ren_color, ms=3, alpha=0.4)
plt.plot(X_con, y, 'o', label='Conventional', color=con_color, ms=3, alpha=0.4)

plt.plot(X_ren, m_ren * X_ren + b_ren, 'black', label='Regression Line (Renewable)', linewidth=2)
plt.plot(X_con, m_con * X_con + b_con, 'black', label='Regression Line (Conventional)', linewidth=2, linestyle='dotted')

plt.xlabel('Electricity Production [MWh]')
plt.ylabel('Price [€/MWh]')

plt.legend()
plt.savefig(os.path.join(fig_src, 'ren_vs_con_regression.pdf'))
plt.show()

# Quarterly Technology Mix

In [ ]:
quarterly       = combined.resample('3M').mean()
quarterly.index = quarterly.index.date

# quarterly = quarterly.drop(columns=['Price', 'Solar & Wind', 'Wind', 'Pumped-Storage Hydro'])
quarterly = quarterly.drop(columns=['Price', 'Solar & Wind', 'Wind'])

fig = quarterly.plot(kind='bar',
              stacked=True,
              width=0.4,
              figsize=(10,5),
              rot=45)

fig.set_xlabel('Date')
fig.set_ylabel('Electricity Generation [MWh]')

plt.savefig(os.path.join(fig_src, 'quarterly_technology_mix.pdf'))
plt.show()